# Project 1: Training a Simple Neural Network with GPU

## Introduction

In this project, you will create, train, and evaluate a simple neural network using both TensorFlow and PyTorch. The objective is to ensure you are comfortable with setting up a neural network and utilizing GPU acceleration for training. You will use the MNIST dataset for this project.

## Objectives

1. Set up TensorFlow and PyTorch environments.
2. Verify GPU availability.
3. Implement a simple neural network in TensorFlow and PyTorch.
4. Train and evaluate the models.
5. Answer assessment questions.

## Instructions

Follow the steps below to complete the project. Ensure that you use a GPU to train your models.

---

### Step 1: Set Up Your Environment

First, install the necessary libraries. Run the following cell to install TensorFlow and PyTorch.


Provide snapshots from your environment showing:
1) You are using a virtual environment
2) You have installed `TensorFlow` and `PyTorch`

---

### Step 2: Verify GPU Availability
Check if TensorFlow and PyTorch can detect the GPU.

Run the following two code blocks and show the output.

#### TensorFlow GPU Check

In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available for TensorFlow!")
else:
    print("No GPU found for TensorFlow.")

TensorFlow version: 2.15.0
GPU is available for TensorFlow!


#### PyTorch GPU Check

In [3]:
import torch

print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU is available for PyTorch!")
else:
    print("No GPU found for PyTorch.")

PyTorch version: 2.3.0+cu121
GPU is available for PyTorch!


---

### Step 3: Implement and Train a Simple Neural Network
#### TensorFlow Implementation
1. Load and preprocess the MNIST dataset.
2. Define the neural network model.
3. Compile the model.
4. Train the model using the GPU.
5. Evaluate the model.

You need to complete and run the code. Show the complete output.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test)= mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
with tf.device('/GPU:0'):
    model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy:.4f}')


11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/5
1688/1688 [==============================] - 8s 4ms/step - loss: 0.2704 - accuracy: 0.9240 - val_loss: 0.1227 - val_accuracy: 0.9693
Epoch 2/5
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1161 - accuracy: 0.9657 - val_loss: 0.0921 - val_accuracy: 0.9747
Epoch 3/5
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0797 - accuracy: 0.9752 - val_loss: 0.0842 - val_accuracy: 0.9753
Epoch 4/5
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0592 - accuracy: 0.9819 - val_loss: 0.0772 - val_accuracy: 0.9782
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0740 - accuracy: 0.9779
Test accuracy: 0.9779


#### PyTorch Implementation
1. Load and preprocess the MNIST dataset.
2. Define the neural network model.
3. Define loss function and optimizer.
4. Train the model using the GPU.
5. Evaluate the model.

You need to complete and run the code. Show the complete output.


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4169647.26it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 133358.10it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 240373.76it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 8612354.78it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Loss: 0.2222
Epoch [2/5], Loss: 0.2132
Epoch [3/5], Loss: 0.1062
Epoch [4/5], Loss: 0.1892
Epoch [5/5], Loss: 0.0273
Test Accuracy: 96.11%


---
### Questions
Answer the following questions in detail.

1. What is the purpose of normalizing the input data in both TensorFlow and PyTorch implementations?

* **Answer:** Normalization ensures that the input data is scaled to a standard range, typically between 0 and 1. This helps accelerate the convergence of the training process. Normalized data provides stable gradients during backpropagation and prevents issues like vanishing or exploding gradients. It ensures that all input features contribute equally to the learning process.


2. Explain the role of the activation function relu in the neural network.
* **Answer:** The ReLU (Rectified Linear Unit) activation function introduces non-linearity into the neural network, allowing it to learn complex patterns. It outputs the input directly if it is positive, otherwise, it outputs zero, which helps in mitigating the vanishing gradient problem. ReLU also activates only a subset of neurons at a time, making the network more efficient. Additionally, it is computationally efficient due to its simple mathematical operation.


3. Why is it important to use GPU for training neural networks?
* **Answer:** Using a GPU for training neural networks is important because it allows for parallel processing, significantly speeding up the computation of matrix and vector operations. GPUs handle the high computational demands of training deep networks more efficiently than CPUs. They reduce the training time considerably, making it feasible to work with large datasets and complex models. Additionally, GPUs provide scalability, essential for handling extensive training processes in deep learning.

4. Compare the training time and accuracy of the TensorFlow and PyTorch models. Which one performed better and why?
* **Answer:**

* **Comparison of Training Time and Accuracy**

The TensorFlow model trained for approximately 8 seconds in the first epoch and 5-7 seconds for each of the subsequent epochs. It achieved a validation accuracy of 98.07% at the end of 5 epochs and a test accuracy of 97.79%. Although the exact training times for the PyTorch model were not provided, they are typically comparable to TensorFlow when using the same hardware and batch sizes due to GPU acceleration. The PyTorch model achieved a test accuracy of 96.11%.


* **Which One Performed Better and Why?**

In terms of accuracy, the TensorFlow model performed better, achieving a higher test accuracy of 97.79% compared to PyTorch's 96.11%. This indicates that the TensorFlow model generalized slightly better on the MNIST dataset for this specific implementation.

Both TensorFlow and PyTorch leverage GPU acceleration, resulting in fast training times. Although the exact training times for PyTorch were not provided, they are typically similar to TensorFlow's when using the same hardware and batch sizes.

The minor difference in accuracy could be due to implementation specifics, such as initial weights, slight variations in training processes, or optimization settings. TensorFlow might have better initial default settings or slight differences in the way it handles certain operations that contributed to the marginally higher accuracy. Additionally, TensorFlow and PyTorch have different internal optimizations, which can affect performance and accuracy slightly.

In conclusion, the TensorFlow model performed better in terms of accuracy, achieving 97.79% compared to PyTorch's 96.11%. Both frameworks are efficient with GPU, and the training times are likely comparable. The specific implementation and possible optimizations within TensorFlow might have contributed to slightly better performance in this instance. It is also possible that the TensorFlow model's training process better handled the dataset characteristics, leading to improved generalization.

---
### Submission
Submit a link to your completed Jupyter Notebook (e.g., on GitHub (private) or Google Colab) with all the cells executed, and answers to the assessment questions included at the end of the notebook.